##Here I experimented with the code of Jason Brownlee

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
!wget https://www.gutenberg.org/files/11/11-0.txt

--2019-08-26 09:09:05--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: ‘11-0.txt.1’

11-0.txt.1          100%[===================>] 169.53K   182KB/s    in 0.9s    

2019-08-26 09:09:07 (182 KB/s) - ‘11-0.txt.1’ saved [173595/173595]



In [0]:
# load ascii text and covert to lowercase
filename = "11-0.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


In [0]:

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '#': 3, '$': 4, '%': 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '?': 25, '@': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55, '‘': 56, '’': 57, '“': 58, '”': 59, '\ufeff': 60}


In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163817
Total Vocab:  61


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  163717


In [8]:
print(n_vocab)

61


In [9]:
print(chars)

['\n', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


In [10]:

len(chars)

61

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
dataX

[[60,
  45,
  47,
  44,
  39,
  34,
  32,
  49,
  1,
  36,
  50,
  49,
  34,
  43,
  31,
  34,
  47,
  36,
  57,
  48,
  1,
  30,
  41,
  38,
  32,
  34,
  57,
  48,
  1,
  30,
  33,
  51,
  34,
  43,
  49,
  50,
  47,
  34,
  48,
  1,
  38,
  43,
  1,
  52,
  44,
  43,
  33,
  34,
  47,
  41,
  30,
  43,
  33,
  9,
  1,
  31,
  54,
  1,
  41,
  34,
  52,
  38,
  48,
  1,
  32,
  30,
  47,
  47,
  44,
  41,
  41,
  0,
  0,
  49,
  37,
  38,
  48,
  1,
  34,
  31,
  44,
  44,
  40,
  1,
  38,
  48,
  1,
  35,
  44,
  47,
  1,
  49,
  37,
  34,
  1,
  50,
  48,
  34,
  1,
  44],
 [45,
  47,
  44,
  39,
  34,
  32,
  49,
  1,
  36,
  50,
  49,
  34,
  43,
  31,
  34,
  47,
  36,
  57,
  48,
  1,
  30,
  41,
  38,
  32,
  34,
  57,
  48,
  1,
  30,
  33,
  51,
  34,
  43,
  49,
  50,
  47,
  34,
  48,
  1,
  38,
  43,
  1,
  52,
  44,
  43,
  33,
  34,
  47,
  41,
  30,
  43,
  33,
  9,
  1,
  31,
  54,
  1,
  41,
  34,
  52,
  38,
  48,
  1,
  32,
  30,
  47,
  47,
  44,
  41,
  41,
  0,


In [13]:

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0826 09:09:12.809967 140199887058816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 09:09:12.833544 140199887058816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 09:09:12.836801 140199887058816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 09:09:13.339580 140199887058816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0826 09:09:13.350917 

In [14]:
# For retrieving the saved best results
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
cd /content/gdrive/My Drive

/content/gdrive/My Drive


In [16]:
mkdir eip_2

mkdir: cannot create directory ‘eip_2’: File exists


In [17]:
cd eip_2

/content/gdrive/My Drive/eip_2


In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/eip_2/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [19]:

model.fit(X, y, epochs=10, batch_size=256, callbacks=callbacks_list)


W0826 09:09:17.025544 140199887058816 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
163717/163717 [==============================] - 125s 765us/step - loss: 3.0512

Epoch 00001: loss improved from inf to 3.05118, saving model to /content/gdrive/My Drive/eip_2/weights-improvement-01-3.0512.hdf5
Epoch 2/10
163717/163717 [==============================] - 124s 758us/step - loss: 2.8829

Epoch 00002: loss improved from 3.05118 to 2.88285, saving model to /content/gdrive/My Drive/eip_2/weights-improvement-02-2.8829.hdf5
Epoch 3/10
163717/163717 [==============================] - 124s 756us/step - loss: 2.8054

Epoch 00003: loss improved from 2.88285 to 2.80544, saving model to /content/gdrive/My Drive/eip_2/weights-improvement-03-2.8054.hdf5
Epoch 4/10
163717/163717 [==============================] - 124s 757us/step - loss: 2.7439

Epoch 00004: loss improved from 2.80544 to 2.74392, saving model to /content/gdrive/My Drive/eip_2/weights-improvement-04-2.7439.hdf5
Epoch 5/10
163717/163717 [==============================] - 123s 752us/step - loss: 2.6926

Epoch 00

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [21]:
import sys
# load the network weights
filename = "/content/gdrive/My Drive/eip_2/weights-improvement-09-2.2811.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

OSError: ignored

###Running with the changes mentioned, for 500 characters and a padded sequence

In [0]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
# load ascii text and covert to lowercase
filename = "11-0.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
raw_text

In [0]:
raw_text1 = raw_text.replace(',','')

In [0]:
raw_text1

In [0]:
print(raw_text1)

In [0]:
from keras.preprocessing.sequence import pad_sequences

def process_data(data):
  corpus= data.splitlines()
  #dataX=[]
  #dataY=[]
  input_sequences=[]
  for line in corpus:
    for i in range(1,len(line)):
      seq=list(line[:i+1])
      input_sequences.append([char_to_int[char] for char in seq])

  max_sequence_len = max([len(x) for x in input_sequences])
  padded_input_sequences=pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
  #dataX.append(padded_input_sequences[:,:-1])
  #dataY.append(padded_input_sequences[:,-1])

  return padded_input_sequences,max_sequence_len

In [0]:

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text1)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
padded_input_sequences,max_sequence_len=process_data(raw_text1)

In [0]:
len(padded_input_sequences)

In [0]:
print(padded_input_sequences[:,:-1][2])

In [0]:
print(padded_input_sequences[:,-1][2])

In [0]:
dataX=[]
dataY=[]
for i in range(1,len(padded_input_sequences)):
  dataX.append(padded_input_sequences[:,:-1][i])
  dataY.append(padded_input_sequences[:,-1][i])


In [0]:
dataX

In [0]:
n_patterns = len(dataX)

In [0]:
n_patterns

In [0]:
chars = sorted(list(set(raw_text1)))
n_vocab = len(chars)

In [0]:
154723*77

In [0]:
len(dataX)

In [0]:
X = numpy.reshape(dataX, (n_patterns, max_sequence_len-1, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/eip_2/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

In [0]:
model.fit(X, y, epochs=54, batch_size=256, callbacks=callbacks_list)

In [0]:
model.load_weights('/content/gdrive/My Drive/eip_2/weights-improvement-07-1.6765.hdf5')

In [0]:
model.fit(X, y, epochs=47, batch_size=256, callbacks=callbacks_list)

In [0]:
import sys
# load the network weights
filename = ""
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")